In [1]:
import pandas as pd
import numpy as np

In [9]:
df = pd.read_csv("anes_timeseries_2020/anes_timeseries_2020_csv_20220210.csv", encoding='latin1')

#don't use any data to train the model where there is no clear answer on the self-reported political ideology
invalid_ideology_values = [-9, -8, 99]
df_valid = df[~df['V201200'].isin(invalid_ideology_values)]


/var/folders/hb/ql93z14s1z15lknblg3cqpsw0000gn/T/ipykernel_34415/2071522146.py:1: DtypeWarning: Columns (15,17,18,19,21,22,23,25,26,27,29,30,31,33,34,35,37,38,1508,1509) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("anes_timeseries_2020/anes_timeseries_2020_csv_20220210.csv", encoding='latin1')


In [14]:
#turn numbers on the scale into labels to avoid creating a sense of distance between categories
def ideology_label(val):
    if val == 1:
        return "Extremely liberal"
    elif val == 2:
        return "Liberal"
    elif val == 3:
        return "Slightly liberal"
    elif val == 4:
        return "Moderate"
    elif val == 5:
        return "Slightly conservative"
    elif val == 6:
        return "Conservative"
    elif val == 7:
        return "Extremely conservative"
    else:
        raise Exception("Value is not on the required scale")

Y = df_valid['V201200'].apply(ideology_label)



0             Conservative
1                 Moderate
2                  Liberal
3         Slightly liberal
4    Slightly conservative
Name: V201200, dtype: object
